In [23]:
# TODO Turn this into a .py file so that the client can run it.

In [24]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

In [25]:
# Be a good neighbor. Let other people use the GPU too. 
# print(tf.config.list_physical_devices('GPU'))
# gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)

In [26]:
# Load up the training data
data_file_location = 'training_data/'
data_file_name = 'data03_train_2500.npy'
label_file_name = 'labels03_train_2500.npy'

checkpoint_filepath = 'temp/checkpoint/convnet03.h5'
log_filepath = 'temp/tb_logs'

data = np.load(data_file_location+data_file_name)
labels = np.load(data_file_location+label_file_name)

In [27]:
x = data
y = labels[:,0]

In [28]:
def build_model(input_shape,output_size):
    model = Sequential()
    model.add(Conv1D(10, kernel_size=5, padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(20, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(30, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(40, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(50, kernel_size=5, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(60, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(70, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(80, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Conv1D(90, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2, strides=2))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dense(output_size, activation='softmax'))
    return model
    

In [29]:
def build_simple_model(input_shape, output_size):
    model = Sequential()
    model.add(Conv1D(1, kernel_size=3, padding='same', input_shape=input_shape, activation='relu'))
    model.add(Flatten())
    model.add(Dense(output_size, activation='softmax'))
    return model

In [30]:
model = build_model(input_shape=(1024,8),output_size=25)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_18 (Conv1D)          (None, 1024, 10)          410       
                                                                 
 max_pooling1d_18 (MaxPooli  (None, 512, 10)           0         
 ng1D)                                                           
                                                                 
 conv1d_19 (Conv1D)          (None, 512, 20)           1020      
                                                                 
 max_pooling1d_19 (MaxPooli  (None, 256, 20)           0         
 ng1D)                                                           
                                                                 
 conv1d_20 (Conv1D)          (None, 256, 30)           3030      
                                                                 
 max_pooling1d_20 (MaxPooli  (None, 128, 30)          

In [31]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [32]:
# Setup the callbacks
checkpt = ModelCheckpoint(checkpoint_filepath, save_best_only=True, verbose=0)
tb = TensorBoard(log_dir=log_filepath)
e_stop = EarlyStopping(patience=5)

In [37]:
# Train the model
hist = model.fit(x, y, epochs=100, validation_split=0.2, verbose=2, batch_size=100,
                 callbacks=[checkpt, tb, e_stop])

Epoch 1/100
20/20 - 1s - loss: 0.0530 - accuracy: 0.9830 - val_loss: 3.0080 - val_accuracy: 0.6320 - 907ms/epoch - 45ms/step
Epoch 2/100
20/20 - 1s - loss: 0.0502 - accuracy: 0.9810 - val_loss: 2.8739 - val_accuracy: 0.6360 - 542ms/epoch - 27ms/step
Epoch 3/100
20/20 - 1s - loss: 0.0445 - accuracy: 0.9840 - val_loss: 2.9990 - val_accuracy: 0.6300 - 516ms/epoch - 26ms/step
Epoch 4/100
20/20 - 0s - loss: 0.0427 - accuracy: 0.9835 - val_loss: 3.1084 - val_accuracy: 0.6220 - 467ms/epoch - 23ms/step
Epoch 5/100
20/20 - 1s - loss: 0.0408 - accuracy: 0.9860 - val_loss: 3.1154 - val_accuracy: 0.6080 - 582ms/epoch - 29ms/step
Epoch 6/100
20/20 - 0s - loss: 0.0381 - accuracy: 0.9885 - val_loss: 3.2186 - val_accuracy: 0.6280 - 471ms/epoch - 24ms/step
Epoch 7/100
20/20 - 1s - loss: 0.0273 - accuracy: 0.9915 - val_loss: 3.4940 - val_accuracy: 0.6120 - 501ms/epoch - 25ms/step
